In [197]:
%%bash
wget -P data/STEP/ https://raw.githubusercontent.com/STEPBible/STEPBible-Data/master/Older%20Formats/TOTHT%20Gen-Deu%20-%20Translators%20OT%20Hebrew%20Tagged%20text%20-%20STEPBible.org%20CC%20BY.txt -o Gen_Deu.txt
wget -P data/STEP/ https://raw.githubusercontent.com/STEPBible/STEPBible-Data/master/Older%20Formats/TOTHT%20Jos-Est%20-%20Translators%20OT%20Hebrew%20Tagged%20text%20-%20STEPBible.org%20CC%20BY.txt -O Jos_Est.txt
wget -P data/STEP/ https://raw.githubusercontent.com/STEPBible/STEPBible-Data/master/Older%20Formats/TOTHT%20Job-Sng%20-%20Translators%20OT%20Hebrew%20Tagged%20text%20-%20STEPBible.org%20CC%20BY.txt -O Job_Sng.txt
wget -P data/STEP/ https://raw.githubusercontent.com/STEPBible/STEPBible-Data/master/Older%20Formats/TOTHT%20Isa-Mal%20-%20Translators%20OT%20Hebrew%20Tagged%20text%20-%20STEPBible.org%20CC%20BY.txt -O Isa_Mal.txt

--2024-09-09 16:39:23--  https://raw.githubusercontent.com/STEPBible/STEPBible-Data/master/Older%20Formats/TOTHT%20Jos-Est%20-%20Translators%20OT%20Hebrew%20Tagged%20text%20-%20STEPBible.org%20CC%20BY.txt
Resolving raw.githubusercontent.com... 2606:50c0:8003::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11979137 (11M) [text/plain]
Saving to: 'Jos_Est.txt'

     0K .......... .......... .......... .......... ..........  0% 2.38M 5s
    50K .......... .......... .......... .......... ..........  0% 18.1M 3s
   100K .......... .......... .......... .......... ..........  1% 4.48M 3s
   150K .......... .......... .......... .......... ..........  1% 9.51M 2s
   200K .......... .......... .......... .......... ..........  2% 10.2M 2s
   250K .......... .......... .......... .......... ..........  2% 6.49M 2s
   300K .......... .......... .......... ....

In [8]:
prefix = '/Users/hope/Documents/Documents - Hope’s MBP (120)/10 Projects/14 PhD/14.01 GitHub/literary-translation/data/STEP/'
paths = ['Gen_Deu.txt', 'Jos_Est.txt', 'Job_Sng.txt', 'Isa_Mal.txt']
paths = [prefix + p for p in paths]
import json
ATNACH = '\u0591'

In [4]:
import sys
sys.path.append('/Users/hope/Documents/Documents - Hope’s MBP (120)/10 Projects/14 PhD/14.01 GitHub/literary-translation/scripts')

In [6]:
# Function to parse the input data and produce the desired output
def get_half_lines(line):
    words = line.split()
    split = None
    for i, w in enumerate(words):
        if ATNACH in w:
            split = i
    if split != None:
        return ' '.join(words[:split+1]), ' '.join(words[split+1:])
    else:
        return line, ''


def parse_data_file(input_file):
    parsed_data = []
    current_ref = None
    current_text = []
    # Open the input file for reading
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

        # Iterate through each line in the file, skipping headers
        for line in lines[3:]:
            columns = line.strip().split('\t')
            if len(columns) < 6:
                continue  # Skip lines that do not have enough data

            heb_ref, eng_ref, pointed, accented, morphology, extended_strongs = columns

            # Extract the Hebrew and English references, ignoring the word number
            heb_verse = heb_ref.split('-')[0]
            eng_verse = eng_ref.split('-')[0]


            # # If we're still processing the same reference, keep appending words
            if current_ref == heb_verse:
                current_text.append(accented.replace('/', ''))
            else:
                # If we encounter a new reference, store the previous one (if it exists)
                if current_ref is not None:
                    line = ' '.join(current_text)
                    half_a, half_b = get_half_lines(line)
                    parsed_data.append({
                        'heb_ref': current_ref,
                        'eng_ref': current_eng_ref,
                        'line': line,
                        'half_a': half_a,
                        'half_b': half_b
                    })
                # Start collecting data for the new reference
                current_ref = heb_verse
                current_eng_ref = eng_verse
                current_text = [accented.replace('/', '')]

        # Append the last collected reference
        if current_ref:
            line = ' '.join(current_text)
            half_a, half_b = get_half_lines(line)
            parsed_data.append({
                'heb_ref': heb_verse,
                'eng_ref': eng_verse,
                'line': line,
                'half_a': half_a,
                'half_b': half_b
            })
    return parsed_data

In [9]:
data = []
for p in paths:
    data.extend(parse_data_file(p))


In [203]:
# Write the output as a JSON file
with open('../data/STEP/OT_aligned.json', 'w', encoding='utf-8') as out_f:
    for line in data:
        json.dump(line, out_f, ensure_ascii=False)
        out_f.write('\n')


In [15]:
import re
def remove_cantillation_marks(text):
    # Define the regex pattern
    # U+05C3 is a sof pasuq, we want to keep that.
    # pattern = r'[\u0591-\u05AF\u05C0-\u05C2\u05C4-\u05C7\u05F3\u05F4]'
    # actually remove sof pasuq
    pattern = r'[\u0591-\u05AF\u05C0-\u05C7\u05F3\u05F4]'

    # Use the sub() function to replace matches with an empty string
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text
def remove_nikkud(text):
    # Define the regex pattern
    return re.sub(r'[\u0591-\u05C7]', '', text)
